# Demo T1 processing notebook

## Startup qMRLab

In [ ]:
try
    cd qMRLab
    startup
    cd ..
catch
    error("qMRLab could not be started correctly.")
end

## Get configuration options



In [ ]:
datasetDetails=loadjson('example.json');
submission = fieldnames(datasetDetails);

dataTypeList = {};
imagePathList = {};
counter = 1;

for ii = 1:length(submission)
    datasetList = fieldnames(datasetDetails.(submission{ii}).datasets);

    for jj = 1:length(datasetList)
        OSF_linkList{counter} = datasetDetails.(submission{ii}).OSF_link;

        siteDetails=datasetDetails.(submission{ii}).datasets.(datasetList{jj});
        dataTypeList{counter} = siteDetails.dataType;
        if strcmp(siteDetails.dataType, 'Complex')
            imagePathList{counter} = siteDetails.magnitude.imagePath;
            counter = counter + 1;
            OSF_linkList{counter} = datasetDetails.(submission{ii}).OSF_link;
            dataTypeList{counter} = siteDetails.dataType;
            imagePathList{counter} = siteDetails.phase.imagePath;
            counter = counter + 1;
        else
            imagePathList{counter} = siteDetails.imagePath;
            counter = counter + 1;
        end
    end
end

## Download data

In [ ]:
downloadedList = {''};
counter = 1;

for ii = 1:length(imagePathList)

    OSF_link= OSF_linkList{ii};

    if ~strcmp(OSF_link, downloadedList)
        % Download data
        download_data(OSF_link);
        
        downloadedList{counter} = OSF_link;
        counter = counter + 1;
    end
    
end

## Convert YAML config files to YAML

In [ ]:
%get dataTypeList --from MATLAB
%get imagePathList --from MATLAB

from convert_yaml_to_json import *

for dataType, imagePath in zip(dataTypeList, imagePathList):
    print(imagePath)
    convert_yaml_to_json(dataType, imagePath)


## Process entire dataset

In [ ]:
disp('Beginning dataset processing phase.')

track_complex = false;

for ii = 1:length(imagePathList)
    disp(['----- Started dataset ', num2str(ii), ' out of ', num2str(length(imagePathList)), ' -----'])

    if track_complex
        track_complex = false;
        disp('Skipping, second file of already processed complex dataset.')
        continue
    end
    dataType = dataTypeList{ii};
    imagePath = imagePathList{ii};

    if strcmp(dataType, 'Complex')
        disp('Complex dataset encountered.')
        % Load data
        [data_mag, hdr_mag] = load_data(imagePath);
        ii = ii+1;
        [data_ph, hdr_ph] = load_data(imagePathList{ii});
        data = magphaseToComplex(data_mag, data_ph);
        hdr = hdr_mag;
        track_complex = true;
    else
        % Load data
        [data, hdr] = load_data(imagePath);
    end

    % Get inversion times
    TI = load_inversion_times(imagePath);

    % Fit data
    FitResults = fit_inversion_recovery(data, TI, dataType);

    % Save data to nifti

    [filepath,name,ext] = fileparts(imagePath);
    if strcmp(name(end-3:end),'.nii')
       [tmp1,name,tmp2] = fileparts(name);
    end
    clear tmp1, tmp2;

    t1MapPath = ['t1_map' filesep filepath];
    if strcmp(dataType, 'Complex')
        t1Filename = [name(1:end-9), 'Complex_t1map.nii.gz'];
    else
        t1Filename = [name, '_t1map.nii.gz'];
    end
    save_to_nifti(FitResults, hdr, t1MapPath, t1Filename)

end

disp('Completed processing entire datasets!')